In [1]:
# import packages
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.express as px
import numpy as np

### Source of info

- **results/HQcl_representative_MAGs_all_info.xlsx** - Representative genomes from HQ clustering. Created with Final_build_HQcl.ipynb (1,641 lines)
- **results/LQcl_representative_MAGs_all_info.xlsx** - Representative genomes from LQ clustering. Created with Final_build_LQcl.ipynb (5 lines)
- **results/HQLQcl_together_Cdb.csv** - info about secondary_cluster. Created with dREP (9,475 lines, only bins goin to clustering)
  
- **input/HQLQcl_representative_genomes_list.csv** - list of representatives genomes from cluster. Created with 'ls * all dereplicated 
  

In [2]:
## Step 1. Combine HQ and LQ representative genomes
# Combine HQcl_representative_MAGs_all_info.xlsx an LQcl_representative_MAGs_all_info.xlsx
HQcl = pd.read_excel('results/HQcl_representative_MAGs_all_info.xlsx')
LQcl = pd.read_excel('results/LQcl_representative_MAGs_all_info.xlsx')
# Combine HQcl and LQcl
HQcl_LQcl = pd.concat([HQcl, LQcl])
# reset index
HQcl_LQcl = HQcl_LQcl.reset_index(drop=True)
HQcl_LQcl.tail(2)


,rep_MAG_name,pre_cluster,completeness,contamination,strain_heterogeneity,length,N50,bst_preCl_bin_name,cluster,domain,phylum,class,order,family,genus,sp,bin_number_in_cluster,bin_qual,rep_MAG_ID
3899,VPOVJ1_maxbin.023.fa,NaN,90.02,40.44,2.7,4085276,1811,NaN,LQ-112_1,Bacteria,Cyanobacteriota,Vampirovibrionia,Gastranaerophilales,Gastranaerophilaceae,Zag111,undefined,1,LQ,L1416_Zag111_undS.fa
3900,VXL3GC_vamb.168970_sub.fa,NaN,65.52,12.07,0.0,1579696,108092,NaN,LQ-385_2,Bacteria,Cyanobacteriota,Vampirovibrionia,Gastranaerophilales,Gastranaerophilaceae,Zag111,undefined,1,LQ,L1588_Zag111_undS.fa


In [3]:
## Step 1. Add info about cluster
cluster = pd.read_csv('input/HQLQcl_together_Cdb.csv')
# rename column genome
cluster = cluster.rename(columns={'genome': 'rep_MAG_ID'})
# keep only bin name and cluster columns
cluster = cluster[['rep_MAG_ID', 'secondary_cluster']]
cluster = cluster.rename(columns={'secondary_cluster': 'HQLQ_cluster'})
cluster.tail(2)

# merge HQcl_LQcl and cluster
mrg_st1 = pd.merge(HQcl_LQcl, cluster, on='rep_MAG_ID', how='outer')
mrg_st1.tail(2)

,rep_MAG_name,pre_cluster,completeness,contamination,strain_heterogeneity,length,N50,bst_preCl_bin_name,cluster,domain,phylum,class,order,family,genus,sp,bin_number_in_cluster,bin_qual,rep_MAG_ID,HQLQ_cluster
3899,VPOVJ1_maxbin.023.fa,NaN,90.02,40.44,2.7,4085276,1811,NaN,LQ-112_1,Bacteria,Cyanobacteriota,Vampirovibrionia,Gastranaerophilales,Gastranaerophilaceae,Zag111,undefined,1,LQ,L1416_Zag111_undS.fa,162_1
3900,VXL3GC_vamb.168970_sub.fa,NaN,65.52,12.07,0.0,1579696,108092,NaN,LQ-385_2,Bacteria,Cyanobacteriota,Vampirovibrionia,Gastranaerophilales,Gastranaerophilaceae,Zag111,undefined,1,LQ,L1588_Zag111_undS.fa,154_1


In [4]:
# Step 2. Read the list of representative genomes from HQLQ clustering
lst = pd.read_csv('input/HQLQcl_representative_genomes_list.csv')
lst.tail(2)

# define cluster for HQLQ_rep_MAGs
mrg_st2 = pd.merge(lst, mrg_st1, left_on='HQLQ_rep_MAG_ID', right_on='rep_MAG_ID', how='inner')
mrg_st2 = mrg_st2[['HQLQ_rep_MAG_ID', 'HQLQ_cluster']]
mrg_st2.tail(2)

# Step 3. merge mrg_st1 and mrg_st2
mrg_st3 = pd.merge(mrg_st1, mrg_st2, on='HQLQ_cluster', how='outer')
mrg_st3 = mrg_st3[['HQLQ_rep_MAG_ID','HQLQ_cluster','rep_MAG_ID', 'bin_qual','completeness', 'contamination',
       'strain_heterogeneity', 'length', 'N50', 'cluster', 'domain', 'phylum', 'class', 
       'order', 'family', 'genus', 'sp', 'bin_number_in_cluster','rep_MAG_name']]
mrg_st3.tail(2)

,HQLQ_rep_MAG_ID,HQLQ_cluster,rep_MAG_ID,bin_qual,completeness,contamination,strain_heterogeneity,length,N50,cluster,domain,phylum,class,order,family,genus,sp,bin_number_in_cluster,rep_MAG_name
3899,L1642_Vescimonas_undS.fa,1900_2,L1642_Vescimonas_undS.fa,LQ,66.77,25.86,41.18,1729642,49503,LQ-752_3,Bacteria,Bacillota_A,Clostridia,Oscillospirales,Oscillospiraceae,Vescimonas,undefined,1,VZTMOW_maxbin.071.fa
3900,L1373_WRAV01_undS.fa,1846_1,L1373_WRAV01_undS.fa,MQ,70.13,0.00,0.00,1449916,3454,LQ-714_1,Bacteria,Bacillota_A,Clostridia,Oscillospirales,Ruminococcaceae,WRAV01,undefined,1,VOHZH4_metabat.225.fa


In [5]:
# Step 4. Caclulate the number of genomes in each HQLQ_cluster
bn = mrg_st3[['rep_MAG_ID','HQLQ_cluster']].groupby('HQLQ_cluster').count()
# rename column
bn = bn.rename(columns={'rep_MAG_ID': 'rep_MAG_ID_in_HQLQcluster'})
# merge it with mrg_st3
mrg_st4 = pd.merge(mrg_st3, bn, on='HQLQ_cluster', how='outer')

mrg_st4.tail()


,HQLQ_rep_MAG_ID,HQLQ_cluster,rep_MAG_ID,bin_qual,completeness,contamination,strain_heterogeneity,length,N50,cluster,domain,phylum,class,order,family,genus,sp,bin_number_in_cluster,rep_MAG_name,rep_MAG_ID_in_HQLQcluster
3896,L1276_Vescimonas_sp900545495_undS.fa,1896_2,L1276_Vescimonas_sp900545495_undS.fa,LQ,63.95,18.34,60.00,1670540,35465,LQ-755_13,Bacteria,Bacillota_A,Clostridia,Oscillospirales,Oscillospiraceae,Vescimonas,Vescimonas sp900545495,1,VL4EV4_metabat.21_sub.fa,1
3897,L1353_Vescimonas_undS.fa,1892_4,L1353_Vescimonas_undS.fa,LQ,85.34,25.86,52.94,2176363,20109,LQ-757_4,Bacteria,Bacillota_A,Clostridia,Oscillospirales,Oscillospiraceae,Vescimonas,undefined,1,VNNCHP_maxbin.082.fa,1
3898,L1516_Vescimonas_undS.fa,1898_1,L1516_Vescimonas_undS.fa,LQ,71.87,27.59,61.90,2283535,29276,LQ-755_6,Bacteria,Bacillota_A,Clostridia,Oscillospirales,Oscillospiraceae,Vescimonas,undefined,1,VTCFL1_vamb.219170_sub.fa,1
3899,L1642_Vescimonas_undS.fa,1900_2,L1642_Vescimonas_undS.fa,LQ,66.77,25.86,41.18,1729642,49503,LQ-752_3,Bacteria,Bacillota_A,Clostridia,Oscillospirales,Oscillospiraceae,Vescimonas,undefined,1,VZTMOW_maxbin.071.fa,1
3900,L1373_WRAV01_undS.fa,1846_1,L1373_WRAV01_undS.fa,MQ,70.13,0.00,0.00,1449916,3454,LQ-714_1,Bacteria,Bacillota_A,Clostridia,Oscillospirales,Ruminococcaceae,WRAV01,undefined,1,VOHZH4_metabat.225.fa,1


Moove to representatives MAGs from cluster

In [6]:
## Step 5. Select only representative MAGs
mrg_st5 = pd.merge(lst, mrg_st4, left_on='HQLQ_rep_MAG_ID', right_on='rep_MAG_ID', how='inner')
# rename column genome
mrg_st5 = mrg_st5.rename(columns={'HQLQ_rep_MAG_ID_x': 'HQLQ_rep_MAG_ID'})
# drop column centrality
mrg_st5 = mrg_st5.drop(columns=['rep_MAG_ID','HQLQ_rep_MAG_ID_y'])
mrg_st5.tail()

,HQLQ_rep_MAG_ID,HQLQ_cluster,bin_qual,completeness,contamination,strain_heterogeneity,length,N50,cluster,domain,phylum,class,order,family,genus,sp,bin_number_in_cluster,rep_MAG_name,rep_MAG_ID_in_HQLQcluster
2723,L1631_Haemophilus_D_parainfluenzae_K.fa,106_1,LQ,100.00,49.29,74.00,3007490,1894,LQ-457_3,Bacteria,Pseudomonadota,Gammaproteobacteria,Enterobacterales_A,Pasteurellaceae,Haemophilus_D,Haemophilus_D parainfluenzae_K,1,VZGZLV_maxbin.027.fa,1
2724,L1632_HGM05190_sp900759815.fa,663_1,MQ,57.05,0.00,0.00,1340836,8501,LQ-501_1,Bacteria,Bacteroidota,Bacteroidia,Bacteroidales,Muribaculaceae,HGM05190,HGM05190 sp900759815,1,VZHO9I_vamb.250220.fa,1
2725,L1638_UMGS872_undS.fa,380_2,LQ,50.00,24.14,0.00,1283430,56641,LQ-185_4,Bacteria,Bacillota,Bacilli,RF39,UBA660,UMGS872,undefined,1,VZR2LZ_maxbin.071_sub.fa,1
2726,L1639_Oxalobacter_undS.fa,2063_1,LQ,73.38,17.45,7.14,2857230,1116,LQ-1022_2,Bacteria,Pseudomonadota,Gammaproteobacteria,Burkholderiales,Burkholderiaceae,Oxalobacter,undefined,1,VZR2LZ_maxbin.165.fa,1
2727,L1642_Vescimonas_undS.fa,1900_2,LQ,66.77,25.86,41.18,1729642,49503,LQ-752_3,Bacteria,Bacillota_A,Clostridia,Oscillospirales,Oscillospiraceae,Vescimonas,undefined,1,VZTMOW_maxbin.071.fa,1


## Save the files

In [34]:
# SAVE FINAL TABLE with all MAGs came from HQ an LQ clustering 
#mrg_st4.to_excel('results/together_HQLQcl_all_MAGs_all_info.xlsx', index=False)

In [7]:
# SAVE FINAL TABLE with only HQLQ_rep_MAGs came from HQ an LQ clustering 
mrg_st5.to_excel('results/together_HQLQcl_representative_MAGs_all_info.xlsx', index=False)

## Make a table with all possible bins!

In [8]:
hq_all = pd.read_excel('results/HQcl_all_bins_all_info.xlsx')
hq_all = hq_all[['bin_name', 'bin_qual', 'completeness', 'contamination','domain', 'phylum', 'class', 
                 'order', 'family', 'genus', 'sp','cluster', 'bin_number_in_cluster','rep_MAG_ID',
                 'strain_heterogeneity', 'length']]

lq_all = pd.read_excel('results/LQcl_all_bins_all_info.xlsx')
lq_all = lq_all[['bin_name', 'bin_qual', 'completeness', 'contamination','domain', 'phylum', 'class', 
                 'order', 'family', 'genus', 'sp','cluster', 'bin_number_in_cluster','rep_MAG_ID',
                 'strain_heterogeneity', 'length']]

bin_all = pd.concat([hq_all, lq_all]).reset_index(drop=True, inplace=True)


In [27]:
bin_all.head(2)

,bin_name,bin_qual,completeness,contamination,domain,phylum,class,order,family,genus,sp,cluster,bin_number_in_cluster,rep_MAG_ID,strain_heterogeneity,length
0,V0404P_175861.fa,MQ,84.48,0.0,Bacteria,Bacteroidota,Bacteroidia,Bacteroidales,Muribaculaceae,Limisoma,Limisoma sp000437795,HQ-613_1,580,H0011_Limisoma_sp000437795.fa,0.0,2184033
1,V1T1XL_vamb.223401.fa,HQ,91.38,0.0,Bacteria,Bacteroidota,Bacteroidia,Bacteroidales,Muribaculaceae,Limisoma,Limisoma sp000437795,HQ-613_1,580,H0011_Limisoma_sp000437795.fa,0.0,2306061


In [28]:
ncl_bin = pd.read_excel('results/LQ_bins_without_cl_b2b3.xlsx')
#ncl_bin = ncl_bin.rename(columns={'rep_MAG_name': 'bin_name'})
ncl_bin.head(2)

,bin_name,domain,phylum,class,order,family,genus,sp
0,V04YHL_vamb.15881.fa,Archaea,Methanobacteriota,Methanobacteria,Methanobacteriales,Methanobacteriaceae,Methanobrevibacter_A,Methanobrevibacter_A smithii
1,V1BPF7_metabat.81.fa,Archaea,Methanobacteriota,Methanobacteria,Methanobacteriales,Methanobacteriaceae,Methanobrevibacter_A,Methanobrevibacter_A smithii


In [29]:
# Merge df1 and df2 to fill undefined values
merged_df = bin_all.merge(ncl_bin, on='bin_name', suffixes=('', '_df2'), how='left')

# Replace undefined values in df1 with values from df2
columns_to_fill = ['domain', 'phylum', 'class','order', 'family', 'genus', 'sp']
for column in columns_to_fill:
    undefined_mask = merged_df[column] == 'undefined'
    merged_df.loc[undefined_mask, column] = merged_df.loc[undefined_mask, f'{column}_df2']

# Drop the extra columns from df2
merged_df.drop(columns=[f'{column}_df2' for column in columns_to_fill], inplace=True)

merged_df.tail(2)


,bin_name,bin_qual,completeness,contamination,domain,phylum,class,order,family,genus,sp,cluster,bin_number_in_cluster,rep_MAG_ID,strain_heterogeneity,length
84760,V5TGN9_vamb.150645_sub.fa,LQ,100.0,120.69,NaN,NaN,NaN,NaN,NaN,NaN,NaN,undefined,5955,NaN,35.65,7065269
84761,V68KXR_55755_sub.fa,LQ,100.0,125.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,undefined,5955,NaN,26.45,10934240


In [33]:
# SAVE FINAL TABLE with all MAGs came from HQ an LQ clustering 
merged_df.to_excel('results/together_all_MAGs_all_info.xlsx', index=False)